In [1]:
#imports
!pip install langchain
!pip install faiss-cpu
!pip install ollama
!pip install openpyxl
!pip install pandas
!pip install requests
!pip install langchain-community langchain-ollama langchain-groq
!pip install langgraph


INFO: pip is looking at multiple versions of langchain-ollama to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of langchain-groq to determine which version is compatible with other requirements. This could take a while.
  Attempting uninstall: langchain-ollama
    Found existing installation: langchain-ollama 0.2.2
    Uninstalling langchain-ollama-0.2.2:
      Successfully uninstalled langchain-ollama-0.2.2
  Attempting uninstall: langchain-groq
    Found existing installation: langchain-groq 0.2.2
    Uninstalling langchain-groq-0.2.2:
      Successfully uninstalled langchain-groq-0.2.2
  Using cached msgpack-1.1.0-cp312-cp312-macosx_11_0_arm64.whl.metadata (8.4 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.8/130.8 kB 2.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.4/45.4 kB 4.0 MB/s eta 0:00:00
Using cached msgpack-1.1.0-cp312-cp312-macosx_11_0_arm64.whl (82 kB)
  A

In [4]:
import os
import json
import requests
import logging
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import OllamaEmbeddings
import pandas as pd


In [5]:

# Logging configuration
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

# Constants
KNOWLEDGE_BASE_DIR = r"C:\Users\Akram\OneDrive\Documents\kiran\stenosis_txt"
JSON_FILE = "final_raw_questions_stenosis.json"
#OUTPUT_FILE = "responses.xlsx"

In [6]:
# Function to load text files from a directory
def load_text_files(directory):
    docs = []
    for filename in os.listdir(directory):
        if filename.endswith(".txt"):
            filepath = os.path.join(directory, filename)
            loader = TextLoader(filepath, encoding="utf-8")
            docs.extend(loader.load())
    return docs

# Load documents
logging.info("Loading documents...")
documents = load_text_files(KNOWLEDGE_BASE_DIR)
logging.info(f"Loaded {len(documents)} documents.")


2025-02-16 20:40:26,930 - INFO - Loading documents...
2025-02-16 20:40:27,047 - INFO - Loaded 10 documents.


In [8]:
# Function to split text into manageable chunks
def split_text_into_chunks(docs, chunk_size=1000, chunk_overlap=200):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    return text_splitter.split_documents(docs)

# Split documents
logging.info("Splitting documents into chunks...")
data_chunks = split_text_into_chunks(documents)
logging.info(f"Created {len(data_chunks)} chunks.")


2025-02-16 20:40:54,974 - INFO - Splitting documents into chunks...
2025-02-16 20:40:54,984 - INFO - Created 49 chunks.


In [ ]:
# Initialize embedding model and create vector store
logging.info("Creating vector store...")
embedding_model = OllamaEmbeddings(model="nomic-embed-text")  # Adjust model if needed
vectorstore = FAISS.from_documents(data_chunks, embedding_model)
vectorstore.save_local("vectore_store")
logging.info("Vector store created.")


2025-02-16 20:42:27,980 - INFO - Creating vector store...
C:\Users\Akram\AppData\Local\Temp\ipykernel_18332\784870715.py:3: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embedding_model = OllamaEmbeddings(model="nomic-embed-text")  # Adjust model if needed
2025-02-16 20:44:28,593 - INFO - Loading faiss with AVX2 support.
2025-02-16 20:44:28,762 - INFO - Successfully loaded faiss with AVX2 support.
2025-02-16 20:44:28,778 - INFO - Failed to load GPU Faiss: name 'GpuIndexIVFFlat' is not defined. Will not load constructor refs for GPU indexes.
2025-02-16 20:44:28,791 - INFO - Vector store created.


In [11]:
# Initialize retriever
vector_store=FAISS.load_local("vectore_store",embedding_model,allow_dangerous_deserialization=True)
retriever = vectorstore.as_retriever(search_kwargs={"k": 2})
logging.info("Retriever initialized.")


2025-02-16 20:47:03,759 - INFO - Retriever initialized.


In [12]:
import re

In [88]:
def load_questions_from_json(json_file):
    with open(json_file, "r") as file:
        raw_questions = json.load(file)
    
    questions = []
    for item in raw_questions:
        question_text = item.get("question", "").strip()
        questions.append({"question": question_text})

    return questions

# Load questions
logging.info("Loading questions...")
questions = load_questions_from_json(JSON_FILE)
logging.info(f"Loaded {len(questions)} questions.")


2025-02-16 22:58:39,284 - INFO - Loading questions...
2025-02-16 22:58:39,289 - INFO - Loaded 2 questions.


In [91]:
from langchain_groq import ChatGroq
from langchain_ollama import ChatOllama
from langchain_core.prompts import PromptTemplate,ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough,RunnableParallel
from langchain.tools import tool
vector_store=FAISS.load_local("vectore_store",embedding_model,allow_dangerous_deserialization=True)
retriever = vectorstore.as_retriever(search_kwargs={"k": 2})
from pydantic import BaseModel, Field
from typing import Literal


def combine_docs(docs):
    combined = "\n\n".join(doc.page_content for doc in docs)
    return combined 


class ResponseSchema(BaseModel):
    option: Literal["opa", "opb", "opc", "opd", "unknown"] = Field(

        description="The selected answer option. Must be one of: 'opa', 'opb', 'opc', 'opd', or 'unknown'."
    )


@tool
def retriver_tool(question:str):
    """ Always use this tool to get medical information"""
    try :
        llm= ChatGroq(
            model="llama-3.3-70b-versatile",
            api_key="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"
        ).with_structured_output(ResponseSchema)

        # llm = ChatOllama(model="llama3.1").with_structured_output(ResponseSchema)

        context_docs=retriever.get_relevant_documents(question)
        context=combine_docs(context_docs)
        prompt = f"""
        You are a medical AI assistant. Answer the following multiple-choice question based on the given context.

        - If the context provides enough information, select the correct answer.
        - If the context does NOT provide enough information, return "unknown".
        - Your response should be only one of: "opa", "opb", "opc", "opd", or "unknown".
        - Do not provide explanations or any extra text.

        {question}  # Full question including options

        Context:
        {context}

        Your answer (choose only "opa", "opb", "opc", "opd", or "unknown"):
        """
        prompt_template= PromptTemplate.from_template(prompt)
        chain=(
            RunnableParallel(context=retriever,question=RunnablePassthrough()) |
            prompt_template |
            llm
        )

        return chain.invoke(question)
    except Exception as e:
        return ResponseSchema(option="unknown")

    


Testing the chain


In [90]:
a=retriver_tool("Carotid artery stenosis screening invivo choice is - opa: USG, opb: CT, opc: MRI, opd: Doppler ")
print(a.option)

2025-02-16 22:59:13,410 - INFO - HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


opd


In [ ]:
import pandas as pd

def save_to_excel(questions, filename="responses.xlsx"):
    data = []
    
    for idx, question_data in enumerate(questions, start=1):

        question = question_data.get("question", "No question provided")
        response = retriver_tool(question)
        data.append({
            "Question": question,  
            "Predicted Answer": response.option  
        })

    df = pd.DataFrame(data)
    df.to_excel(filename, index=False)
    logging.info(f"Saved results to {filename}")

save_to_excel(questions)


2025-02-16 22:56:27,029 - INFO - HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
2025-02-16 22:56:32,661 - INFO - HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
2025-02-16 22:56:32,777 - INFO - Saved results to responses.xlsx


This is Agent Keep it as backup


In [65]:
from langchain_core.messages import SystemMessage,HumanMessage
from langchain_core.prompts.chat import MessagesPlaceholder,HumanMessagePromptTemplate
from langgraph.prebuilt import create_react_agent
from langchain.agents import create_tool_calling_agent,AgentExecutor


llm= ChatGroq(
            model="llama-3.1-8b-instant",
            api_key="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"
        )

agent_prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content="""You are a medical assistant designed to provide answers in the required output format.

            - Always use the tool for answering medical-related questions.
            - Your response should be only one of: "opa", "opb", "opc", "opd", or "unknown".
            - Do not provide explanations or any extra text.
            """
        ),
        HumanMessagePromptTemplate.from_template("{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad")
    ]
)

medical_agent=create_tool_calling_agent(
    llm,
    tools=[retriver_tool],
    prompt=agent_prompt
)
agent_executor = AgentExecutor(
    agent=medical_agent,
    tools=[retriver_tool],
    verbose=True
)

In [66]:
print(agent_executor.invoke({"input":"Carotid artery stenosis screening invivo choice is - opa: USG, opb: CT, opc: MRI, opd: Doppler"}))



> Entering new AgentExecutor chain...


2025-02-16 21:44:15,198 - INFO - HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"



Invoking: `retriver_tool` with `{'question': 'What is the best in vivo screening choice for carotid artery stenosis?'}`




2025-02-16 21:44:20,723 - INFO - HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


unknown

...

unknown

2025-02-16 21:45:42,080 - INFO - HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"



Invoking: `retriver_tool` with `{'question': 'Carotid artery stenosis in vivo screening choices'}`




2025-02-16 21:45:47,325 - INFO - HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


unknown

> Finished chain.
{'input': 'Carotid artery stenosis screening invivo choice is - opa: USG, opb: CT, opc: MRI, opd: Doppler', 'output': 'Agent stopped due to max iterations.'}


In [ ]:
def ollama_llm_strict(question, context):
    url = "http://localhost:11434/api/chat"
    
    prompt = f"""
You are a medical AI assistant. Answer the following multiple-choice question based on the given context.

- If the context provides enough information, select the correct answer.
- If the context does NOT provide enough information, return "unknown".
- Your response should be only one of: "opa", "opb", "opc", "opd", or "unknown".
- Do not provide explanations or any extra text.

{question}  # Full question including options

Context:
{context}

Your answer (choose only "opa", "opb", "opc", "opd", or "unknown"):
"""
    
    data = {
        "model": "llama3.1:8b",  # Adjust model as needed
        "temperature": 0,
        "messages": [{"role": "user", "content": prompt}]
    }
    
    try:
        response = requests.post(url, json=data)
        response.raise_for_status()

        # Process the response line by line to handle streaming JSON
        raw_text = response.text.strip().split("\n")  # Split streamed JSON lines
        final_response = ""

        for line in raw_text:
            try:
                json_line = json.loads(line)  # Parse each line as JSON
                if "message" in json_line and "content" in json_line["message"]:
                    final_response += json_line["message"]["content"].strip().lower()
            except json.JSONDecodeError:
                continue  # Skip invalid JSON chunks

        logging.info(f"Final extracted response: {final_response}")

        valid_options = {"opa", "opb", "opc", "opd", "unknown"}

        # Extract valid answer
        if not final_response:
            return "unknown"

        processed_response = final_response.split()[0] if final_response.split() else "unknown"
        
        return processed_response if processed_response in valid_options else "unknown"

    except requests.exceptions.RequestException as e:
        logging.error(f"Error querying Ollama API: {e}")
        return "unknown"

In [18]:
!pip install langchain-groq

In [117]:
import pandas as pd

def save_to_excel(questions, filename="responses.xlsx"):
    data = []
    
    for idx, question_data in enumerate(questions, start=1):
        question = question_data.get("question", "No question provided")

        # Retrieve relevant context
        retrieved_docs = retriever.get_relevant_documents(question)
        context = combine_docs(retrieved_docs)
        
        # Query LLM for the answer (we no longer pass options separately)
        response = ollama_llm_strict(question, context)

        # Append data for Excel
        data.append({
            "Question": question,  
            "Predicted Answer": response  # Only store the predicted answer
        })

    # Convert to DataFrame
    df = pd.DataFrame(data)
    
    # Save to Excel
    df.to_excel(filename, index=False)
    logging.info(f"Saved results to {filename}")

# Run function to save results
save_to_excel(questions)


2025-02-12 01:55:33,261 - INFO - Final extracted response: opd
2025-02-12 01:55:35,339 - INFO - Final extracted response: opb
2025-02-12 01:55:37,588 - INFO - Final extracted response: opb
2025-02-12 01:55:39,588 - INFO - Final extracted response: opd
2025-02-12 01:55:42,070 - INFO - Final extracted response: opb
2025-02-12 01:55:43,956 - INFO - Final extracted response: opc
2025-02-12 01:55:46,210 - INFO - Final extracted response: opb
2025-02-12 01:55:48,311 - INFO - Final extracted response: opa
2025-02-12 01:55:50,313 - INFO - Final extracted response: opd
2025-02-12 01:55:52,530 - INFO - Final extracted response: opa
2025-02-12 01:55:54,505 - INFO - Final extracted response: opa
2025-02-12 01:55:56,512 - INFO - Final extracted response: opc
2025-02-12 01:55:58,472 - INFO - Final extracted response: opa
2025-02-12 01:56:00,437 - INFO - Final extracted response: opc
2025-02-12 01:56:02,410 - INFO - Final extracted response: unknown
2025-02-12 01:56:04,522 - INFO - Final extracted re